In [1]:
# Template Notebook for Indvidualized Participant Engagement Classifcation NN Model

# ToDo/Goals: https://docs.google.com/document/d/1_dTnbdRoBV0A7xCxVEyLBdckrlsUnMnRFbbRn39tiXk/edit?usp=sharing
# Data: https://drive.google.com/drive/folders/19aJUAlkTMz7PcZE1q4hogFkjVtwYGcMT
# Upload code to help-seeking github: https://github.com/interaction-lab/help_seeking
# Record model results here: https://docs.google.com/spreadsheets/d/16ye54fSSEuAuDL_j56UIeDB-rIIrxq_kPbtyPRQOrVI/edit?usp=sharing

In [2]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics

from keras.layers import Dense, Activation
from keras.models import Sequential

Using TensorFlow backend.


#### TODO: Specifiy Data Path Based on Participant/Your Computer

In [4]:
# TODO: Specify Data Path based on participant/your computer
# USE data from FS4 folder

file = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p8_data_FS4.csv'
data = pd.read_csv(file)

In [5]:
data.head()

,of_confidence,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,...,ts_attempt,skill_NC,skill_OS,skill_EM,diff_1,diff_2,diff_3,diff_4,diff_5,no_game
0,0.0,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,...,0.0,0,0,0,0,0,0,0,0,1
1,0.0,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,...,0.0,0,0,0,0,0,0,0,0,1
2,0.0,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,...,0.0,0,0,0,0,0,0,0,0,1
3,0.0,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,...,0.0,0,0,0,0,0,0,0,0,1
4,0.0,0.0,0.49709,0.500431,0.519632,0.51422,0.502452,0.531622,0.588569,0.514524,...,0.0,0,0,0,0,0,0,0,0,1


In [6]:
# Label Analysis

print(data['engagement'].value_counts())
print()

print(1,data['engagement'].value_counts()[1]/sum(data['engagement'].value_counts()))
print(0,data['engagement'].value_counts()[0]/sum(data['engagement'].value_counts()))

1.0    139463
0.0    120473
Name: engagement, dtype: int64

1 0.5365282223316509
0 0.4634717776683491


### Train Test Split (for chosen Feature Set)

#### TODO: Specify which train-test split you want

1. Random Shuffle Split: 70-30, 50-50, 30-70
2. Chronological Split (80-20, 70-30, 60-40, 50-50, 40-60, 30-70, 20-80)

In [7]:
# Sort data (don't need for Random Shuffle, but ok to leave it)

data = data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [8]:
# TODO: Uncomment correct train-test split below

y = data['engagement']
X = data.drop(columns=['timestamp', 'session_num', 'engagement'])

# Choose one of these 

# 1. Random Shuffle Split 70-30
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Random Shuffle Split 50-50
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# 3. Random Shuffle Split 30-70
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

#############################

# 4. Chronological Split 80-20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 5. Random Shuffle Split 70-30
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

# 6. Random Shuffle Split 60-40
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)

# 7. Random Shuffle Split 50-50
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)

# 8. Random Shuffle Split 40-60
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, shuffle=False)

# 9. Random Shuffle Split 30-70
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, shuffle=False)

# 10. Random Shuffle Split 20-80
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, shuffle=False)


### Step 3: Neural Network Model

#### TODO: May need to adjust following if poor accurracy
1. Vary num_epochs, batch size
2. Vary between 2, 3, 4 hidden layers
3. Vary amount of nodes in each layer

In [9]:
num_epochs = 10          # if bad, try 5, 10, 15, 20 
size_batch = 128         # if bad, try 64, 128, 256 

In [10]:
# NN Model for FS4 (all OF/OP)

model = Sequential()

model.add(Dense(units=64, input_dim=len(X_train.columns)))
model.add(Activation('relu'))

model.add(Dense(units=32))
model.add(Activation('relu'))

model.add(Dense(units=1))
model.add(Activation('sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                1920      
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
_________________________________________________________________
activation_3 (Activation)    (None, 1)                 0         
Total params: 4,033
Trainable params: 4,033
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='binary_crossentropy',  
              optimizer='Adam',
              metrics=['accuracy'])

### 4. Model Evaluation

In [12]:
model.fit(X_train, y_train, epochs=num_epochs, batch_size=size_batch, verbose=1)

Epoch 1/10
207948/207948 [==============================] - 3s 14us/step - loss: 0.3757 - acc: 0.8278
Epoch 2/10
207948/207948 [==============================] - 3s 12us/step - loss: 0.2868 - acc: 0.8781
Epoch 3/10
207948/207948 [==============================] - 3s 12us/step - loss: 0.2541 - acc: 0.8940
Epoch 4/10
207948/207948 [==============================] - 3s 13us/step - loss: 0.2340 - acc: 0.9022
Epoch 5/10
207948/207948 [==============================] - 3s 13us/step - loss: 0.2182 - acc: 0.9084
Epoch 6/10
207948/207948 [==============================] - 3s 13us/step - loss: 0.2058 - acc: 0.9144
Epoch 7/10
207948/207948 [==============================] - 3s 12us/step - loss: 0.1968 - acc: 0.9183
Epoch 8/10
207948/207948 [==============================] - 3s 13us/step - loss: 0.1888 - acc: 0.9219
Epoch 9/10
207948/207948 [==============================] - 3s 13us/step - loss: 0.1818 - acc: 0.9251
Epoch 10/10
207948/207948 [==============================] - 3s 13us/step - loss: 

In [13]:
loss_and_metrics = model.evaluate(X_test, y_test)
print(loss_and_metrics)

51988/51988 [==============================] - 1s 16us/step
[1.5424589053107673, 0.7198968992844502]


In [14]:
scores = model.predict_proba(X_test)
pred = model.predict_classes(X_test)

#### TODO: RECORD ACCURACY IN MODEL RESULTS SPREADSHEET
Try to get accurracy, AUC above 90%!!!

In [15]:
try:
    print("AUC:",roc_auc_score(y_test, scores)) 
except:
    print("AUC undefinied, only 1 class in test data")
    
print("Accuracy:",accuracy_score(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))
print(metrics.classification_report(y_test, pred))

AUC undefinied, only 1 class in test data
Accuracy: 0.7198968992844502
[[37426 14562]
 [    0     0]]
              precision    recall  f1-score   support

         0.0       1.00      0.72      0.84     51988
         1.0       0.00      0.00      0.00         0

   micro avg       0.72      0.72      0.72     51988
   macro avg       0.50      0.36      0.42     51988
weighted avg       1.00      0.72      0.84     51988



/anaconda3/envs/caispp/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
